# Imports

In [1]:
data_loc = "../../../../data/" 

In [2]:
import prophet
import sys

sys.modules['fbprophet'] = prophet

/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import warnings
#warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm
import time

import plotly.offline as pyo
import plotly.graph_objs as go

import itertools
import os, sys

from supersmoother import SuperSmoother, LinearSmoother
from sklearn.preprocessing import StandardScaler


from collections import OrderedDict
sys.path.append("..")
from configs.functions import make_future, add_cond_trend_version, run_prophet_funct, hparam_tuning
from configs.functions import preprocess_data
from configs.bad_direction_kpi_dict import bad_direction_kpi_dict
from configs.kpi_constraints_dict import kpi_constraints_dict

from uuid import uuid4

import dask
from dask.distributed import Client

import logging, sys
#logging.disable(sys.maxsize) #turn off prophet infos

In [4]:
file = "4weeks-subs_mcc-anon.csv"#"4weeks-subs_mcc-anon.csv"
datas = pd.read_csv(data_loc+ file) #pd.read_csv( "../Data/" + file )
print("-"*30,"DF READ ✔️","-"*30)

------------------------------ DF READ ✔️ ------------------------------


# Dask client init

In [5]:
client = Client( dashboard_address=':44594', n_workers = 5, threads_per_worker = 2) #scheduler_address=':37243'
client

2022-06-30 14:44:27,236 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/eaxxprx/Desktop/Work/EEA Code/aio_distributed/aio-distributed/scalability/aio/Non-seasonal-trend/dask-worker-space/worker-cftix6h9', purging
2022-06-30 14:44:27,238 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/eaxxprx/Desktop/Work/EEA Code/aio_distributed/aio-distributed/scalability/aio/Non-seasonal-trend/dask-worker-space/worker-qf3hizzk', purging
2022-06-30 14:44:27,240 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/eaxxprx/Desktop/Work/EEA Code/aio_distributed/aio-distributed/scalability/aio/Non-seasonal-trend/dask-worker-space/worker-6jgwgk2s', purging
2022-06-30 14:44:27,242 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/eaxxprx/Desktop/Work/EEA Code/aio_distributed/aio-distributed/scalability/aio/Non-seasonal-trend/dask-worker-space/worker-i19ng1bj', purging
2022-06-30 14:44:27,244 - distri

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44594/status,
Dashboard: http://127.0.0.1:44594/status,Workers: 5
Total threads: 10,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64348,Workers: 5
Dashboard: http://127.0.0.1:44594/status,Total threads: 10
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:64369,Total threads: 2
Dashboard: http://127.0.0.1:64371/status,Memory: 3.20 GiB
Nanny: tcp://127.0.0.1:64354,


# Parameter setup, metadata import

In [6]:
metadata_store = pd.read_csv(data_loc + "metadata-anon.csv")#pd.read_csv('../Data/metadata_anon.csv')
# Mixed datatypes in the dimension_name col: floats and OrderedDict as str (need eval)

# Get rid of all irrelevant metadata
metadata_store = metadata_store[ metadata_store.model_type == 'non_seasonal_trend' ]
metadata_store = metadata_store[ metadata_store.path == file ]

# evaluate str to OrderedDict
metadata_store.dimension_name = metadata_store.dimension_name.map(str).map(lambda element: eval(element))

#params
missing_data_percentage_param = 0.3
DAILY_FOURIER_ORDER = 0
WEEKLY_FOURIER_ORDER = 0
is_weekend = False
country_name = 'USA'
percent = 0.1
scores = ['mae'] #['mdape', 'mape', 'smape', 'mae']
predictions_write_to = ''
errors_write_to = ''
write_to = ''
alpha = 1.0


#infos
end = pd.to_datetime(metadata_store['ts'].values[0], unit='s')
ts = metadata_store['ts'].values[0]
start = end - pd.Timedelta(4, unit = 'w') # 
files = metadata_store['path'].unique() # arr of unique files

/var/folders/gl/27x4g9zj7kvft9gr77z3dps40000gn/T/ipykernel_79311/83288175.py:1: DtypeWarning:

Columns (3,10) have mixed types. Specify dtype option on import or set low_memory=False.



# Functions

In [7]:
def submit_training(df, row, p_uuid_df):
    """
    Submit every model for training in dask.

    df - timeseries dataframe in prophet ready format
    row - metadata row
    p_uuid_df - dataframe containing all parameter-dict and id
    """
    df_p_dict = {} # store and return performance metrics of models
    
    # submit tasks to the client
    for p_uuid, params_row in p_uuid_df.iterrows(): # we use all_params2 here, which have the weekend param too
        #try:
        params = params_row.to_dict()
        df_p = hparam_tuning(df, params, row,
                                is_weekend = True,
                                parallel = "dask",
                                daily_fourier_order=DAILY_FOURIER_ORDER,
                                weekly_fourier_order=WEEKLY_FOURIER_ORDER)
        #_params = { **params, **{"weekend": True}}
        df_p_dict[ p_uuid ] = df_p

        
    return df_p_dict
    

def resubmit_training(df, row, params_row):

    """
    Submit one model for training in dask
    """

    params = params_row.to_dict()

    df_p = hparam_tuning(df, params, row,
                            is_weekend = True,
                            parallel = "dask",
                            daily_fourier_order=DAILY_FOURIER_ORDER,
                            weekly_fourier_order=WEEKLY_FOURIER_ORDER)

    return df_p


def myForecast(df, tuning_results,
                    end, row, score,
                    daily_fourier_order = 0,
                    weekly_fourier_order = 0,
                    is_weekend = False, 
                    country_name = "USA"):

    """
    df - dataframe with ts and kpi value (y)
    tuning_results - one rowed df with model metrics as cols, 
                    scores belonging to one parameter combination as vals
    row - row of metadata
    end - pd.Timestamp end of an interval of something #? end of interval for known data
    score - str, name of score
    """
    kpi = row["kpi_name"]
    dim_dict = row["dimension_name"]

    # Choosing the best model. If there are multiple equally good, pick one randomly
    tuning_results[score+'_rank'] = tuning_results[score].rank()
    tuning_results['rank'] = tuning_results[score+'_rank']
    params = tuning_results.loc[ tuning_results["rank"].idxmin(), ["changepoint_prior_scale", "changepoint_range"] ].to_dict()
    # Fit model with best params, predict future
    m = run_prophet_funct(df, params, daily_fourier_order, weekly_fourier_order, is_weekend, country_name)
    future = make_future(m, end, 168)
    forecast = m.predict(future)

    # Setting bounds 
    df["doy"] = df.ds.dt.dayofyear
    iqr = (df[['doy', 'y']].groupby('doy').quantile(0.75)-df[['doy', 'y']].groupby('doy').quantile(0.25)).median().values[0]
    minimum = -iqr
    maximum = +iqr

    lower = kpi_constraints_dict[kpi][0]
    upper = kpi_constraints_dict[kpi][1]


    #? What happens here?
    additive_condition = add_cond_trend_version(forecast, percent, lower, upper, minimum, maximum, end)
    if additive_condition:

        if len(m.changepoints[np.abs(np.nanmean(m.params['delta'], axis=0)) >= 0.01].values)==0:
            last_changepoint = start

        else:
            last_changepoint = m.changepoints[np.abs(np.nanmean(m.params['delta'], axis=0)) >= 0.01].values[-1]

        last_point = ((forecast.set_index('ds')[last_changepoint:]['trend']+alpha*minimum>lower)
                    & (forecast.set_index('ds')[last_changepoint:]['trend']+alpha*maximum<upper))[::-1].idxmax() 

        forecast.loc[forecast['ds']>last_point, 'trend'] = forecast.loc[forecast['ds']==last_point, 'trend'].values[0]

        forecast['yhat'] = forecast['trend']

    # Throw away out of bound predictions
    forecast['yhat'] = forecast['yhat'].clip(lower = lower, upper = upper)

    scaler =  StandardScaler(with_mean = False) # RobustScaler
    scaler.fit(df['y'].values.reshape(-1,1)) 

    df = df.set_index("ds")
    forecast = forecast.set_index("ds")

    results = pd.DataFrame( index = forecast.index,
                            columns = ["kpi_name", "dimension_name", "ground_truth",
                                        "pred", "error", "trend", "gt_wo_trend", "pred_wo_trend"])

    results["kpi_name"] = [kpi] * len(results)
    results["dimension_name"] = [dim_dict] * len(results)
    results["ground_truth"] = df.y
    results["pred"] = forecast.yhat
    results["error"] = scaler.transform((df['y']-forecast['yhat']).values.reshape(-1,1)).T[0]
    results["trend"] = forecast.trend
    results["gt_wo_trend"] = df.y - forecast.trend              # ground truth without trend
    results["pred_wo_trend"] = forecast.yhat - forecast.trend   # predictions without trend

    future_results = results.loc[ ~results.index.isin(df.index)]
    past_results = results.loc[ df.index ]

    return past_results #,future_results

# Tracking training status

In [8]:
metadata_store["failed"] = [False] * len(metadata_store)
metadata_store["num_fails"] = [0] * len(metadata_store)
metadata_store["last_failed_ts"] = [None] * len(metadata_store)
metadata_store["ts_uuid"] = metadata_store["failed"].map(lambda x : str(uuid4()))

# Hyperparameters

In [9]:
param_grid = {  'changepoint_prior_scale': [0.01, 0.1, 1.0],
                        'changepoint_range': [0.8, 0.9, 0.95]       }
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

In [10]:
p_uuid_df = pd.DataFrame( { str(uuid4()): p  for p in all_params}).T   # p_uuid: 1 id per parameter combination  for tracking - same for all models with the same hparams, regardless of timeseries

p_uuid_df.iloc[0].to_dict()
for idx, row in p_uuid_df.head().iterrows():
    print(idx, row.to_dict())

c60e0bbd-8f19-42fd-b9f8-5a604bca3aa5 {'changepoint_prior_scale': 0.01, 'changepoint_range': 0.8}
f56936fe-c683-4e6d-b4c2-27cf4942a4a7 {'changepoint_prior_scale': 0.01, 'changepoint_range': 0.9}
2d503c16-9d73-4dcd-8331-fd5f77901ca9 {'changepoint_prior_scale': 0.01, 'changepoint_range': 0.95}
f4e56123-420d-4773-a35b-d04d58df06cc {'changepoint_prior_scale': 0.1, 'changepoint_range': 0.8}
90cd4dcb-2967-496d-8978-964f6e92c6b4 {'changepoint_prior_scale': 0.1, 'changepoint_range': 0.9}


In [11]:
p_uuid_df.head()

,changepoint_prior_scale,changepoint_range
c60e0bbd-8f19-42fd-b9f8-5a604bca3aa5,0.01,0.80
f56936fe-c683-4e6d-b4c2-27cf4942a4a7,0.01,0.90
2d503c16-9d73-4dcd-8331-fd5f77901ca9,0.01,0.95
f4e56123-420d-4773-a35b-d04d58df06cc,0.10,0.80
90cd4dcb-2967-496d-8978-964f6e92c6b4,0.10,0.90


In [12]:
task_status = lambda : pd.Series([ [ fut.status for p_uuid, fut in compute[ts_uuid].items()] for ts_uuid in compute.keys() ]).explode().value_counts()
failed_tasks = lambda : [[ (ts_uuid,p_uuid) for p_uuid, fut in compute[ts_uuid].items() if fut.status != "finished"] for ts_uuid in compute.keys() ]

# Compute

In [13]:
# Preprocessing dataframes, storing them by timeseries id

df_dict = {}
for  _, mdrow in metadata_store.iterrows():
    df = preprocess_data( datas, mdrow, start, end )
    df_dict[ mdrow.ts_uuid ] = df

In [14]:
taskgroups = {}

for  _, mdrow in metadata_store.iterrows():
    df = df_dict[ mdrow.ts_uuid ]

    taskgroup = dask.delayed(submit_training)( df, mdrow, p_uuid_df)
    taskgroups[ mdrow.ts_uuid ] = ( taskgroup )

compute = dask.compute( taskgroups )[0]
time.sleep(20) # compute finishes before all submitted task, wait a little

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 7 forecasts with cutoffs betwe

Initial log joint probability = -7.93187
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       601.068   1.26982e-05       499.921      0.7955      0.7955      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     140       601.075   8.76707e-09       484.213      0.8127      0.8127      177   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
Initial log joint probability = -12.2734
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       594.748   1.66245e-06       480.538           1           1      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     130       594.751   8.00472e-09       513.546      0.2541      0.2541      164   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below to

INFO:prophet:Applying in parallel with <Client: 'tcp://127.0.0.1:64348' processes=5 threads=10, memory=16.00 GiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Applying in parallel with <Client: 'tcp://127.0.0.1:64348' processes=5 threads=10, memory=16.00 GiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

     212       620.708   5.34101e-09       477.256      0.2336      0.2336      271   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
Initial log joint probability = -9.76106
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        581.86   0.000192979       505.875      0.9193      0.9193      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Initial log joint probability = -37.4742
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     155       581.889   8.67933e-09       436.263      0.4452      0.4452      204   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
Initial log joint probability = -12.2734
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       512.362   3.07972e-05  

INFO:prophet:Making 3 forecasts with cutoffs between 2021-02-18 08:00:00 and 2021-02-22 08:00:00
INFO:prophet:Applying in parallel with <Client: 'tcp://127.0.0.1:64348' processes=5 threads=10, memory=16.00 GiB>
INFO:prophet:Making 7 forecasts with cutoffs between 2021-02-17 05:00:00 and 2021-02-23 05:00:00
INFO:prophet:Applying in parallel with <Client: 'tcp://127.0.0.1:64348' processes=5 threads=10, memory=16.00 GiB>
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
INFO:prophet:Making 3 forecasts with cutoffs between 2021-02-18 08:00:00 and 2021-02-22 08:00:00
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use panda

...

/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

      99       529.618    9.4824e-06       25.7892           1           1      131   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     115        529.62   2.70574e-05       28.8557   6.765e-07       0.001      202  LS failed, Hessian reset 
      58        529.56    0.00256921       42.0142   5.889e-05       0.001      108  LS failed, Hessian reset 
      99       529.632   2.22084e-06        41.033      0.1768           1      161   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     152       529.621   1.01367e-07        33.458           1           1      258   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
     114       529.632   9.23203e-09        37.517      0.0636           1      182   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

Initial log joint probability = -9.18937
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       570.051      0.020505       4.73801           1           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       570.482    0.00859308       8.22995           1           1      258   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     269       570.497    6.2671e-05        2.6597   1.646e-05       0.001      391  LS failed, Hessian reset 
     299       570.499   6.35994e-05        2.7008           1           1      425   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     390       570.509   5.08101e-05       3.25792   1.842e-05       0.001      579  LS failed, Hessian reset 
     399       570.509   8.98198e-06       1.96019           1           1      590   
    Iter      log pro

/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


In [16]:
task_status() # possible task status: finished, pending, cancelled, lost, error, newly-created, closing, connecting, running, closed
# finished is good, the others are bad statusesu

finished    288
dtype: int64

# Recomputing unfinished tasks

In [17]:
zookeeper_table = pd.DataFrame(columns = ["ts_uuid", "p_uuid", "ts", "kpi", "dim", "params", "failed", "num_failed", "last_failed_ts"]) 
# here ts in ts_uuid stands for timeseries, while ts is timestamp. 🦆
# failed contains the status of the last retry
zookeeper_table

,ts_uuid,p_uuid,ts,kpi,dim,params,failed,num_failed,last_failed_ts


In [18]:
def refresh_zk_table( zookeeper_table, compute ):
    status_df = pd.DataFrame(compute).applymap(lambda x: x.status)
    unfinished_tasks = [(status_df.index[x], status_df.columns[y]) for x, y in zip(*np.where(status_df.values != 'finished'))] 

    for _, row in zookeeper_table.iterrows():
        ts_uuid = row.ts_uuid
        p_uuid  = row.p_uuid
        
        if (p_uuid, ts_uuid) not in unfinished_tasks:
            mask = ((ts_uuid == zookeeper_table.ts_uuid) & (p_uuid == zookeeper_table.p_uuid))

            zk_row = row.copy()
            zk_row["failed"] = False
            zookeeper_table[ mask ] = zk_row

    for p_uuid, ts_uuid in unfinished_tasks:

        if (ts_uuid in zookeeper_table.ts_uuid.values) and (p_uuid in zookeeper_table.p_uuid.values):

            mask = ((ts_uuid == zookeeper_table.ts_uuid) & (p_uuid == zookeeper_table.p_uuid))
            zk_row = zookeeper_table[ mask ].copy()
            zk_row.num_failed += 1
            zk_row.last_failed_ts = int(time.time())

            zk_row.bad_status = compute[ts_uuid][p_uuid].status
            zookeeper_table[ mask ] = zk_row

        else:
            mdrow = metadata_store[ metadata_store.ts_uuid == ts_uuid ].copy()

            task_to_retry = { "ts_uuid": ts_uuid, "p_uuid": p_uuid, "ts": mdrow.ts.values[0], 
                                "kpi": mdrow.kpi_name.values[0], "dim": mdrow.dimension_name.values[0],
                                "params": p_uuid_df.loc[p_uuid].to_dict(), "failed": True,
                                "num_failed": 1, "last_failed_ts": int(time.time()),
                                "bad_status": compute[ts_uuid][p_uuid].status }
            zookeeper_table = zookeeper_table.append(pd.Series(task_to_retry), ignore_index=True)
            
    return zookeeper_table

In [19]:
def retry_unfinished_tasks( compute ): 
            
    status_df = pd.DataFrame(compute).applymap(lambda x: x.status)
    unfinished_tasks = [(status_df.index[x], status_df.columns[y]) for x, y in zip(*np.where(status_df.values != 'finished'))]  # tuple of (p_uuid, ts_uuid)

    re_taskgroup = {}

    for p_uuid, ts_uuid in unfinished_tasks:
        df = df_dict[ ts_uuid ]
        row = metadata_store[ metadata_store.ts_uuid == ts_uuid ]
        params_row = p_uuid_df.loc[ p_uuid ]

        if ts_uuid not in re_taskgroup:
            re_taskgroup[ts_uuid] = {}
        re_taskgroup[ts_uuid][p_uuid] = dask.delayed(resubmit_training)( df, row, params_row )

    re_compute = dask.compute(re_taskgroup)[0] # recompute previously unfinished tasks

    # Putting back new tasks - they must be mostly finished, there might be still unfinished though, then we need to 
    for ts_uuid, tg in re_compute.items():
        
        for p_uuid, task in tg.items():
            compute[ts_uuid][p_uuid] = re_compute[ts_uuid][p_uuid]
        
    return compute

In [20]:
task_status()

finished    288
dtype: int64

In [21]:
bad_status = ["pending", "cancelled", "lost", "error", "newly-created", "closing", "connecting", "running", "closed" ]

for i in range(10):
    
    print(i)
    print(task_status())

    zookeeper_table = refresh_zk_table(zookeeper_table, compute)


    #check if there is an unfinished task
    if not any(status in task_status() for status in bad_status):
        break

    status_df = pd.DataFrame(compute).applymap(lambda x: x.status)
    unfinished_tasks = [(status_df.index[x], status_df.columns[y]) for x, y in zip(*np.where(status_df.values != 'finished'))]

    compute = retry_unfinished_tasks(compute)

    
    
    time.sleep(0.5 * len(unfinished_tasks))

0
finished    288
dtype: int64


In [24]:
status_df = pd.DataFrame(compute).applymap(lambda x: x.status)
finished_tasks = [(status_df.index[x], status_df.columns[y]) for x, y in zip(*np.where(status_df.values == 'finished'))]  # tuple of (p_uuid, ts_uuid)

results = {}

for p_uuid, ts_uuid in finished_tasks: 
    
    if ts_uuid not in results:
        results[ts_uuid] = {}

    results[ts_uuid][p_uuid] = compute[ts_uuid][p_uuid].result()

In [25]:
for ts, taskgroup in results.items():
    for p_uuid, task in taskgroup.items():
        params = p_uuid_df.loc[ p_uuid ]

        
        results[ts][p_uuid] = pd.concat([task.T, params]).T

In [26]:
concated_results  = { ts_uuid : pd.concat([ v for k,v in results[ ts_uuid ].items() ]).reset_index(drop=True) for ts_uuid in results }

# Forecast

In [27]:
detrended_timeseries = {}
for _,mdrow in metadata_store.iterrows():
    df = df_dict[ mdrow.ts_uuid ]
    tuning_results = concated_results[ mdrow.ts_uuid ]

    past_ts = dask.delayed(myForecast)(df, tuning_results, end, mdrow, "mae")
    detrended_timeseries[mdrow.ts_uuid] = past_ts

detrended_timeseries = dask.compute(detrended_timeseries)[0]

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=Tru

Initial log joint probability = -7.93187
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Initial log joint probability = -37.4742
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Initial log joint probability = -9.58351
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Initial log joint probability = -3.90065
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Initial log joint probability = -9.76106
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      97       601.441    6.2263e-09       503.133      0.3214      0.3214      124   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
      99       513.579   3.87226e-07       496.435      0.2866      0.6814      131   
    Iter      log prob        ||dx||      ||grad||      

/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

     199           701    0.00249368       5.30241      0.7866     0.07866      243   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199           701    0.00249368       5.30241      0.7866     0.07866      243   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     599       655.307     0.0111809       11.9217       7.071      0.7071      729   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       799.086   2.73434e-06       37.2707      0.1917      0.6976      131   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     113       799.086   2.07279e-08       38.4265      0.1571       0.821      151   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
     699       655.322   0.000764233       3.56484           1           1      853   
    

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True t

...

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


     606       681.757   3.87266e-05       2.24617   1.121e-05       0.001     1136  LS failed, Hessian reset 
     441       681.732   0.000142947        3.9844   2.274e-05       0.001      816  LS failed, Hessian reset 
     199       530.683    0.00143478       4.49263      0.4023      0.4023      274   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Initial log joint probability = -2.0202
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     494        681.74   0.000262157       2.84233   6.961e-05       0.001      919  LS failed, Hessian reset 
     499        681.74   6.12906e-05       2.12767      0.4107           1      925   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     655       681.758   2.90568e-05       3.14056   9.023e-06       0.001     1238  LS failed, Hessian reset 
     287       530.772   0.000180304       3.53821   2.321e-05    

/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaste

     442       3623.66   6.26991e-07       646.057   1.407e-09       0.001      622  LS failed, Hessian reset 
     808       681.818   9.64787e-05       3.58985   3.316e-05       0.001     1515  LS failed, Hessian reset 
     484       3623.66   2.70942e-07        424.59   6.224e-10       0.001      722  LS failed, Hessian reset 
     497       3623.66   2.94309e-08       352.256      0.4179           1      739   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
     899       681.822   7.96453e-05       3.26454      0.9014      0.5097     1644   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Initial log joint probability = -8.66353
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     912       681.823   5.00272e-05       3.95974   1.807e-05       0.001     1701  LS failed, Hessian reset 
     940       681.823   1.59453e-06       2.61344 

/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/eaxxprx/miniconda3/envs/dprophet/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
